## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [575]:
lng = 'fr'
acteur = 'inesss'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [576]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [577]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'Type', 'text']]
    return df

In [578]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 2035 documents.


In [579]:
base_path = '../03-corpus/2-data/'
folder_path = path.join(base_path, '1-' + lng, acteur)

data.to_csv(folder_path + '.csv')
data

,Corpus,Sous-corpus,Title,text
0,inesss,Biologie médicale et génomique,INESSS: Biologie médicale et génomique,Biologie médicale et génomique Processus et cr...
1,inesss,Biologie médicale et génomique,INESSS: Processus et critères d'évaluation,Biologie médicale et génomique Processus et cr...
2,inesss,Biologie médicale et génomique,INESSS: Processus synchronisé d'évaluation des...,Biologie médicale et génomique Processus et cr...
3,inesss,Biologie médicale et génomique,<span>Soutien au diagnostic moléculaire<//span>,Biologie médicale et génomique Processus et cr...
4,inesss,Biologie médicale et génomique,<span>Avis au ministre<//span>,Biologie médicale et génomique Processus et cr...
...,...,...,...,...
26,inesss,Traumatologie,5,Accueil Continuum de services en traumatologie...
27,inesss,Traumatologie,Lire plus,Sondage - Qu’avez-vous pensé de ce guide : Ind...
28,inesss,Traumatologie,Lire plus,Notice Santécom: 127161 Sondage – Qu’avez-vous...
29,inesss,Traumatologie,"Les caractéristiques, l’historique et l’implan...",Le continuum de services en traumatologie (CST...


### **Nettoyage**

In [580]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [581]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 2035 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [582]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [583]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [584]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 1014373 tokens.
Le POS tagging devrait prendre environ 1 minutes.


In [585]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [586]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [587]:
tagged

[['risques', 'NOM'],
 ["d'", 'PRP'],
 ['hospitalisation', 'NOM'],
 ['et', 'KON'],
 ['projections', 'NOM'],
 ['des', 'PRP:det'],
 ['besoins', 'NOM'],
 ['hospitaliers', 'ADJ'],
 ['traitements', 'NOM'],
 ['spécifiques', 'ADJ'],
 ['à', 'PRP'],
 ['la', 'DET:ART'],
 ['covid-19', 'NOM'],
 ['26', 'NUM'],
 ['réponses', 'NOM'],
 ['rapides', 'ADJ'],
 ['disponibles', 'ADJ'],
 ['affections', 'NOM'],
 ['post-covid-19', 'ADJ'],
 ['(', 'PUN'],
 ['covid-19', 'NOM'],
 ['longue', 'ADJ'],
 [')', 'PUN'],
 ['deux', 'NUM'],
 ['états', 'NOM'],
 ['des', 'PRP:det'],
 ['connaissances', 'NOM'],
 ['STOP', 'NOM'],
 ['deux', 'NUM'],
 ['réponses', 'NOM'],
 ['rapides', 'ADJ'],
 ['STOP', 'NOM'],
 ['un', 'DET:ART'],
 ['STOP', 'NOM'],
 ['et', 'KON'],
 ['huit', 'NUM'],
 ['outils', 'NOM'],
 ['cliniques', 'ADJ'],
 ['STOP', 'NOM'],
 ['deux', 'NUM'],
 ['guides', 'NOM'],
 ['et', 'KON'],
 ['normes', 'NOM'],
 ['disponibles', 'ADJ'],
 ['alternatives', 'ADJ'],
 ['de', 'PRP'],
 ['traitements', 'NOM'],
 ['en', 'PRP'],
 ['contexte', 

### **Lemmatisation** (FrenchLefffLemmatizer)
Sur pause ; peut-être qu'il vaudrait mieux le faire plus tard (au moment d'identifier si chaque terme existe déjà dans la taxo ou non) afin d'éviter de complexifier toutes les autres étapes

In [588]:
# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

#### **Mapping POS Tags** (FRMG)
On doit mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)
#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [589]:
# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [590]:
# df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
# df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
# df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

In [591]:
# tagged_dict = df_tagged.to_dict('records')

In [592]:
# # tokens = df_tagged['Token'].tolist()
# # tags = df_tagged['Tag Lefff'].tolist()
# # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

# for term in tagged_dict:
#     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
#         term_l = lemmatizer.lemmatize(word=term['Token'])
    
#     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
#         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

#     else:
#         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

#     term['Lemme'] = term_l

In [593]:
# tagged_dict

In [594]:
# tagged = DataFrame(tagged_dict)
# tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()

# tagged

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [595]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [596]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 7605556 ngrammes.


In [597]:
ngrammes

[(['risques', 'NOM'], ["d'", 'PRP']),
 (['risques', 'NOM'], ["d'", 'PRP'], ['hospitalisation', 'NOM']),
 (['risques', 'NOM'],
  ["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON']),
 (['risques', 'NOM'],
  ["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON'],
  ['projections', 'NOM']),
 (['risques', 'NOM'],
  ["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON'],
  ['projections', 'NOM'],
  ['des', 'PRP:det']),
 (['risques', 'NOM'],
  ["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON'],
  ['projections', 'NOM'],
  ['des', 'PRP:det'],
  ['besoins', 'NOM']),
 (['risques', 'NOM'],
  ["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON'],
  ['projections', 'NOM'],
  ['des', 'PRP:det'],
  ['besoins', 'NOM'],
  ['hospitaliers', 'ADJ']),
 (["d'", 'PRP'], ['hospitalisation', 'NOM']),
 (["d'", 'PRP'], ['hospitalisation', 'NOM'], ['et', 'KON']),
 (["d'", 'PRP'],
  ['hospitalisation', 'NOM'],
  ['et', 'KON'],
  ['projections', 'NOM']),
 (["d'", 'PRP'],
  ['h

### **Extraction des patrons syntaxiques**

In [598]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [599]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

In [600]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [601]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [602]:
frequencies

FreqDist({('STOP',): 59172, ('de',): 52582, (')',): 28108, ('(',): 28021, ('la',): 21044, ("d'",): 20764, ('.',): 19667, ("l'",): 19387, ('des',): 18290, ('et',): 17764, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [603]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [604]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [605]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [606]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [607]:
phrases = filter_len(phrases)

In [608]:
phrases

[[('risques', "d'", 'hospitalisation'), ['NOM', 'PRP', 'NOM']],
 [('risques', "d'", 'hospitalisation', 'et', 'projections'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM']],
 [('risques', "d'", 'hospitalisation', 'et', 'projections', 'des', 'besoins'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM', 'PRP:det', 'NOM']],
 [('risques',
   "d'",
   'hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ']],
 [('hospitalisation', 'et', 'projections'), ['NOM', 'KON', 'NOM']],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM']],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins', 'hospitaliers'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ']],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ', 'NOM']],
 [('hospitalisation',
   'et',
   'projection

On retire les n-grammes qui apparaissent moins de 30 fois dans le corpus

In [609]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 30]

In [610]:
phrases = filter_freq(phrases)

In [611]:
phrases

[[('risques', "d'", 'hospitalisation'), ['NOM', 'PRP', 'NOM']],
 [('risques', "d'", 'hospitalisation', 'et', 'projections'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM']],
 [('risques', "d'", 'hospitalisation', 'et', 'projections', 'des', 'besoins'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM', 'PRP:det', 'NOM']],
 [('risques',
   "d'",
   'hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers'),
  ['NOM', 'PRP', 'NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ']],
 [('hospitalisation', 'et', 'projections'), ['NOM', 'KON', 'NOM']],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM']],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins', 'hospitaliers'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ']],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements'),
  ['NOM', 'KON', 'NOM', 'PRP:det', 'NOM', 'ADJ', 'NOM']],
 [('hospitalisation',
   'et',
   'projection

In [612]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [613]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [614]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 330075 occurrences de ngrammes.


In [615]:
phrases[:15]

[[('risques', "d'", 'hospitalisation'), 'NOM PRP NOM', 339],
 [('risques', "d'", 'hospitalisation', 'et', 'projections'),
  'NOM PRP NOM KON NOM',
  279],
 [('risques', "d'", 'hospitalisation', 'et', 'projections', 'des', 'besoins'),
  'NOM PRP NOM KON NOM PRP:det NOM',
  279],
 [('risques',
   "d'",
   'hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers'),
  'NOM PRP NOM KON NOM PRP:det NOM ADJ',
  279],
 [('hospitalisation', 'et', 'projections'), 'NOM KON NOM', 279],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins'),
  'NOM KON NOM PRP:det NOM',
  279],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins', 'hospitaliers'),
  'NOM KON NOM PRP:det NOM ADJ',
  279],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements'),
  'NOM KON NOM PRP:det NOM ADJ NOM',
  193],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements',
   'spé

In [616]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('risques', "d'", 'hospitalisation'), 'NOM PRP NOM', 339],
 [('risques', "d'", 'hospitalisation', 'et', 'projections'),
  'NOM PRP NOM KON NOM',
  279],
 [('risques', "d'", 'hospitalisation', 'et', 'projections', 'des', 'besoins'),
  'NOM PRP NOM KON NOM PRP:det NOM',
  279],
 [('risques',
   "d'",
   'hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers'),
  'NOM PRP NOM KON NOM PRP:det NOM ADJ',
  279],
 [('hospitalisation', 'et', 'projections'), 'NOM KON NOM', 279],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins'),
  'NOM KON NOM PRP:det NOM',
  279],
 [('hospitalisation', 'et', 'projections', 'des', 'besoins', 'hospitaliers'),
  'NOM KON NOM PRP:det NOM ADJ',
  279],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements'),
  'NOM KON NOM PRP:det NOM ADJ NOM',
  193],
 [('hospitalisation',
   'et',
   'projections',
   'des',
   'besoins',
   'hospitaliers',
   'traitements',
   'spé

In [617]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 4422 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [618]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:50] # Pour prendre les 50 structures syntaxiques les plus fréquentes dans les MeSH

In [619]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [620]:
terms = filter_patterns(phrases)
#terms = phrases

In [621]:
terms

[[('risques', "d'", 'hospitalisation'), 'NOM PRP NOM', 339],
 [('projections', 'des', 'besoins'), 'NOM PRP:det NOM', 288],
 [('projections', 'des', 'besoins', 'hospitaliers'),
  'NOM PRP:det NOM ADJ',
  288],
 [('besoins', 'hospitaliers'), 'NOM ADJ', 410],
 [('besoins', 'hospitaliers', 'traitements'), 'NOM ADJ NOM', 193],
 [('traitements', 'spécifiques'), 'NOM ADJ', 193],
 [('traitements', 'spécifiques', 'à', 'la', 'covid-19'),
  'NOM ADJ PRP DET:ART NOM',
  193],
 [('réponses', 'rapides'), 'NOM ADJ', 1939],
 [('réponses', 'rapides', 'disponibles'), 'NOM ADJ ADJ', 1737],
 [('affections', 'post-covid-19'), 'NOM ADJ', 221],
 [('covid-19', 'longue'), 'NOM ADJ', 193],
 [('états', 'des', 'connaissances'), 'NOM PRP:det NOM', 199],
 [('réponses', 'rapides'), 'NOM ADJ', 1939],
 [('normes', 'disponibles'), 'NOM ADJ', 193],
 [('normes', 'disponibles', 'alternatives'), 'NOM ADJ ADJ', 193],
 [('traitements', 'en', 'contexte'), 'NOM PRP NOM', 195],
 [('traitements', 'en', 'contexte', 'de', 'pandémi

In [622]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 54 % des termes
On avait 330075 ngrammes, on en a maintenant 150354.


In [623]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [624]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [625]:
terms_patterns

[{'Expression': ('risques', "d'", 'hospitalisation'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence': 339},
 {'Expression': ('projections', 'des', 'besoins'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence': 288},
 {'Expression': ('projections', 'des', 'besoins', 'hospitaliers'),
  'Structure syntaxique': 'NOM PRP:det NOM ADJ',
  'Fréquence': 288},
 {'Expression': ('besoins', 'hospitaliers'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 410},
 {'Expression': ('besoins', 'hospitaliers', 'traitements'),
  'Structure syntaxique': 'NOM ADJ NOM',
  'Fréquence': 193},
 {'Expression': ('traitements', 'spécifiques'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 193},
 {'Expression': ('traitements', 'spécifiques', 'à', 'la', 'covid-19'),
  'Structure syntaxique': 'NOM ADJ PRP DET:ART NOM',
  'Fréquence': 193},
 {'Expression': ('réponses', 'rapides'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence': 1939},
 {'Expression': ('réponses', 'rapides', 'disponibles')

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [626]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [627]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 150354 ngrammes.


In [628]:
frequencies

FreqDist({('STOP',): 59172, ('de',): 52582, (')',): 28108, ('(',): 28021, ('la',): 21044, ("d'",): 20764, ('.',): 19667, ("l'",): 19387, ('des',): 18290, ('et',): 17764, ...})

In [629]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [630]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.001 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [631]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [632]:
significant_coll


[{'Terme': ('risques', "d'", 'hospitalisation'),
  'Structure syntaxique': 'NOM PRP NOM',
  'Fréquence (TF)': 339,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('projections', 'des', 'besoins'),
  'Structure syntaxique': 'NOM PRP:det NOM',
  'Fréquence (TF)': 288,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('projections', 'des', 'besoins', 'hospitaliers'),
  'Structure syntaxique': 'NOM PRP:det NOM ADJ',
  'Fréquence (TF)': 288,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('besoins', 'hospitaliers'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence (TF)': 410,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('besoins', 'hospitaliers', 'traitements'),
  'Structure syntaxique': 'NOM ADJ NOM',
  'Fréquence (TF)': 193,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('traitements', 'spécifiques'),
  'Structure syntaxique': 'NOM ADJ',
  'Fréquence (TF)': 193,
  'LLR': -inf,
  'p-value': 1.0},
 {'Terme': ('traitements', 'spécifiques', 'à', 'la', 'covid-19'),
  'Structure syntaxique': 'NOM ADJ 

In [633]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [634]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
145634,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00
11579,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00
68386,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00
120417,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00
50721,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00
...,...,...,...,...,...
86695,infection par le vih,NOM PRP DET:ART NOM,31,563.909485,1.186260e-124
57585,québec (cmq),NOM PUN NOM PUN,31,226.623082,3.249492e-51
23460,preuves du contexte réel,NOM PRP:det NOM ADJ,31,594.049986,3.295983e-131
23459,preuves du contexte,NOM PRP:det NOM,31,431.563844,7.421965e-96


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [635]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [636]:
max_df = round(0.95 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 95% des documents 
min_df = round(0.01 * nb_docs)         # Pour rejeter les termes qui se retrouvent dans moins de 1% des documents
max_df

1933

In [637]:
min_df

20

In [638]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

1004

In [639]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{}

In [640]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > min_df} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [641]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [642]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,attente d'évaluation,44
1,bicarbonate de sodium,974
2,réponses rapides,193
3,cours d'évaluation,57
4,réponses rapides disponibles,193
...,...,...
1437,données cliniques,27
1438,connaissance inesss,34
1439,composantes du système,28
1440,produits de connaissance inesss,34


In [643]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [644]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193
...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31


In [645]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [646]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
# kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

# ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [647]:
# ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [648]:
# extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
# kwic = {w : [] for w in kw} 

In [649]:
# for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
#     for w in kw:
#         if t.startswith(w):
#             kwic[w].append(t)

In [650]:
# kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [651]:
# for term in kw:
#     df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
#     df_kw.sort_values(["Fréquence (TF)"], 
#         axis=0,
#         ascending=[False], 
#         inplace=True)

#     df_kw.insert(0, 'Mot-clé', term)
#     extrant = concat([extrant, df])


# extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# # file_path = '../04-filtrage/output/'
# # file_path = path.join(file_path, acteur, tag)


# # extrant.to_csv(file_path + '_KWIC' +'.csv')

# extrant

### **Extraction de termes MeSH**

In [652]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44,False
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,False
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193,False
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57,False
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193,False
...,...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41,False
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31,False
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28,False
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31,False


In [653]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [654]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [655]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44,False
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,False
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193,False
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57,False
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193,False
...,...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41,False
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31,False
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28,False
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31,False


In [656]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44,False
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,True
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193,False
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57,False
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193,False
...,...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41,False
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31,False
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28,False
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31,False


In [657]:
# Test - ne retenir uniquement que les termes qui sont des MeSH ? 
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,True
43,services sociaux,NOM ADJ,844,-inf,1.000000e+00,571,True
49,cancer du poumon,NOM PRP:det NOM,533,-inf,1.000000e+00,109,True
72,soins palliatifs,NOM ADJ,323,-inf,1.000000e+00,106,True
82,soins intensifs,NOM ADJ,280,-inf,1.000000e+00,220,True
...,...,...,...,...,...,...,...
1412,facteurs de risque,NOM PRP NOM,33,443.158847,2.223131e-98,26,True
1427,résonance magnétique,NOM ADJ,32,691.662686,1.944121e-152,22,True
1425,lymphome de hodgkin,NOM PRP NOM,32,634.733712,4.669695e-140,22,True
1423,qualité des soins,NOM PRP:det NOM,32,317.805644,4.353961e-71,41,True


### **Extraction de termes existant dans la taxonomie**

In [658]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44,False,False
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,True,False
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193,False,False
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57,False,False
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193,False,False
...,...,...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41,False,False
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31,False,False
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28,False,False
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31,False,False


In [659]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [660]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,attente d'évaluation,NOM PRP NOM,2112,-inf,1.000000e+00,44,False,False
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,True,False
2,réponses rapides,NOM ADJ,1939,-inf,1.000000e+00,193,False,False
3,cours d'évaluation,NOM PRP NOM,1859,-inf,1.000000e+00,57,False,False
4,réponses rapides disponibles,NOM ADJ ADJ,1737,-inf,1.000000e+00,193,False,False
...,...,...,...,...,...,...,...,...
1433,connaissance guide,NOM NOM,31,251.305842,1.348209e-56,41,False,False
1431,programme québécois,NOM ADJ,31,386.410416,5.004299e-86,31,False,False
1430,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,395.266703,5.906485e-88,28,False,False
1429,coup d'œil,NOM PRP NOM,31,710.797862,1.341725e-156,31,False,False


In [661]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_16256\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
1,bicarbonate de sodium,NOM PRP NOM,1956,-inf,1.000000e+00,974,True,False
112,hypertension artérielle,NOM ADJ,200,-inf,1.000000e+00,114,True,False
145,médicaments biosimilaires,NOM ADJ,191,-inf,1.000000e+00,88,True,False
159,qualité de vie,NOM PRP NOM,157,-inf,1.000000e+00,106,True,False
183,point de vue,NOM PRP NOM,132,-inf,1.000000e+00,123,True,False
213,dermatite atopique,NOM ADJ,114,-inf,1.000000e+00,48,True,False
242,troubles neurocognitifs,NOM ADJ,105,-inf,1.000000e+00,97,True,False
243,infections urinaires,NOM ADJ,104,-inf,1.000000e+00,92,True,False
265,aide médicale,NOM ADJ,99,1166.152399,1.384889e-255,91,True,False
269,médicaments génériques,NOM ADJ,97,627.811616,1.495465e-138,88,True,False


In [662]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df['Corpus'] = acteur
df = df[["Corpus", "Terme", "Structure syntaxique",	"Fréquence (TF)", "Fréquence documentaire (DF)", "LLR", "isMeSHTerm", "isTaxoTerm" ]]
df.to_csv(output_path)
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm
0,inesss,attente d'évaluation,NOM PRP NOM,2112,44,-inf,False,False
1,inesss,bicarbonate de sodium,NOM PRP NOM,1956,974,-inf,True,False
2,inesss,réponses rapides,NOM ADJ,1939,193,-inf,False,False
3,inesss,cours d'évaluation,NOM PRP NOM,1859,57,-inf,False,False
4,inesss,réponses rapides disponibles,NOM ADJ ADJ,1737,193,-inf,False,False
...,...,...,...,...,...,...,...,...
1433,inesss,connaissance guide,NOM NOM,31,41,251.305842,False,False
1431,inesss,programme québécois,NOM ADJ,31,31,386.410416,False,False
1430,inesss,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,28,395.266703,False,False
1429,inesss,coup d'œil,NOM PRP NOM,31,31,710.797862,False,False


### **Add MeSH ID & en/fr correspondance from extracted terms**

In [663]:
df['MeSHID'] = None

In [664]:
df

,Corpus,Terme,Structure syntaxique,Fréquence (TF),Fréquence documentaire (DF),LLR,isMeSHTerm,isTaxoTerm,MeSHID
0,inesss,attente d'évaluation,NOM PRP NOM,2112,44,-inf,False,False,None
1,inesss,bicarbonate de sodium,NOM PRP NOM,1956,974,-inf,True,False,None
2,inesss,réponses rapides,NOM ADJ,1939,193,-inf,False,False,None
3,inesss,cours d'évaluation,NOM PRP NOM,1859,57,-inf,False,False,None
4,inesss,réponses rapides disponibles,NOM ADJ ADJ,1737,193,-inf,False,False,None
...,...,...,...,...,...,...,...,...,...
1433,inesss,connaissance guide,NOM NOM,31,41,251.305842,False,False,None
1431,inesss,programme québécois,NOM ADJ,31,31,386.410416,False,False,None
1430,inesss,composantes du système de santé,NOM PRP:det NOM PRP NOM,31,28,395.266703,False,False,None
1429,inesss,coup d'œil,NOM PRP NOM,31,31,710.797862,False,False,None


In [665]:
# Lire le fichier XML contenant les MeSH pour tenter de matecher nos termes avec les MeSH labels

In [666]:
from xml.etree import cElementTree as elemtree
from datetime import date

"""
Use this to parse XML from MeSH (Medical Subject Headings). More information 
on the format at: http://www.ncbi.nlm.nih.gov/mesh
End users will primarily want to call the `parse_mesh` function and do something
with the output.
"""

def parse_mesh(filename):
    """Parse a mesh file, successively generating
    `DescriptorRecord` instance for subsequent processing."""
    for _evt, elem in elemtree.iterparse(filename):
        if elem.tag == 'DescriptorRecord':
            yield DescriptorRecord.from_xml_elem(elem)

def date_from_mesh_xml(xml_elem):
    year = xml_elem.find('./Year').text
    month = xml_elem.find('./Month').text
    day = xml_elem.find('./Day').text
    return date(int(year), int(month), int(day))

class PharmacologicalAction(object):
    """A pharmacological action, denoting the effects of a MeSH descriptor."""
    
    def __init__(self, descriptor_ui):
        self.descriptor_ui = descriptor_ui
    
    @classmethod
    def from_xml_elem(cls, elem):
        descriptor_ui = elem.find('./DescriptorReferredTo/DescriptorUI')
        return cls(descriptor_ui)

class SlotsToNoneMixin(object):
    def __init__(self, **kwargs):
        for attr in self.__slots__:
            setattr(self, attr, kwargs.get(attr, None))
    
    def __repr__(self):
        attrib_repr = ', '.join(u'%s=%r' % (attr, getattr(self, attr)) for attr in self.__slots__)
        return self.__class__.__name__ + '(' + attrib_repr + ')'

class Term(SlotsToNoneMixin):
    """A term from within a MeSH concept."""

    __slots__ = ('term_ui', 'string', 'is_concept_preferred', 'is_record_preferred',
      'is_permuted', 'lexical_tag', 'date_created', 'thesaurus_list')
    
    @classmethod
    def from_xml_elem(cls, elem):
        term = cls()
        term.is_concept_preferred = elem.get('ConceptPreferredYN', None) == 'Y'
        term.is_record_preferred = elem.get('RecordPreferredYN', None) == 'Y'
        term.is_permuted = elem.get('IsPermutedTermYN', None) == 'Y'
        term.lexical_tag = elem.get('LexicalTag')
        for child_elem in elem:
            if child_elem.tag == 'TermUI':
                term.ui = child_elem.text
            elif child_elem.tag == 'String':
                term.name = child_elem.text
            elif child_elem.tag == 'DateCreated':
                term.date_created = date_from_mesh_xml(child_elem)
            elif child_elem.tag == 'ThesaurusIDlist':
                term.thesaurus_list = [th_elem.text for th_elem in child_elem]
        return term

class SemanticType(SlotsToNoneMixin):
    __slots__ = ('ui', 'name')
    
    @classmethod
    def from_xml_elem(cls, elem):
        sem_type = cls()
        for child_elem in elem:
            if child_elem.tag == 'SemanticTypeUI':
                sem_type.ui = child_elem.text
            elif child_elem.tag == 'SemanticTypeName':
                sem_type.name = child_elem.text

class Concept(SlotsToNoneMixin):
    """A concept within a MeSH Descriptor."""
    __slots__ = ( 'ui', 'name', 'is_preferred', 'umls_ui', 'casn1_name', 'registry_num', 
      'scope_note', 'sem_types', 'terms')
    
    @classmethod
    def from_xml_elem(cls, elem):
        concept = cls()
        concept.is_preferred = elem.get('PreferredConceptYN', None) == 'Y'
        for child_elem in elem:
            if child_elem.tag == 'ConceptUI':
                concept.ui = child_elem.text
            elif child_elem.tag == 'ConceptName':
                concept.name = child_elem.find('./String').text
            elif child_elem.tag == 'ConceptUMLSUI':
                concept.umls_ui
            elif child_elem.tag == 'CASN1Name':
                concept.casn1_name = child_elem.text
            elif child_elem.tag == 'RegistryNumber':
                concept.registry_num = child_elem.text
            elif child_elem.tag == 'ScopeNote':
                concept.scope_note = child_elem.text
            elif child_elem.tag == 'SemanticTypeList':
                concept.sem_types = [SemanticType.from_xml_elem(st_elem)
                  for st_elem in child_elem.findall('SemanticType')]
            elif child_elem.tag == 'TermList':
                concept.terms = [Term.from_xml_elem(term_elem)
                  for term_elem in child_elem.findall('Term')]
        return concept

class DescriptorRecord(SlotsToNoneMixin):
    "A MeSH Descriptor Record."""
    
    __slots__ = ('ui', 'name', 'date_created', 'date_revised', 'pharm_actions', 
      'tree_numbers', 'concepts')
    
    @classmethod
    def from_xml_elem(cls, elem):
        rec = cls()
        for child_elem in elem:
            if child_elem.tag == 'DescriptorUI':
                rec.ui = child_elem.text
            elif child_elem.tag == 'DescriptorName':
                rec.name = child_elem.find('./String').text
            elif child_elem.tag == 'DateCreated':
                rec.date_created = date_from_mesh_xml(child_elem)
            elif child_elem.tag == 'DateRevised':
                rec.date_revised = date_from_mesh_xml(child_elem)
            elif child_elem.tag == 'TreeNumberList':
                rec.tree_numbers = [tn_elem.text
                  for tn_elem in child_elem.findall('TreeNumber')]
            elif child_elem.tag == 'ConceptList':
                rec.concepts = [Concept.from_xml_elem(c_elem) 
                  for c_elem in child_elem.findall('Concept')]
            elif child_elem.tag == 'PharmacologicalActionList':
                rec.pharm_actions = [PharmacologicalAction.from_xml_elem(pa_elem) 
                  for pa_elem in child_elem.findall('PharmacologicalAction')]
        return rec

                  

In [667]:
file_path = '../04-filtrage/MeSH/fredesc2019.xml'
with open(file_path,  encoding='utf-8') as f:
    xml = read_xml(f, encoding='utf-8')

xml

XMLSyntaxError: Document is empty, line 1, column 1 (<string>, line 1)

In [ ]:
def parse_mesh(filename):
    """Parse a mesh file, successively generating
    `DescriptorRecord` instance for subsequent processing."""
    for _evt, elem in elemtree.iterparse(filename):
        if elem.tag == 'DescriptorRecord':
            yield DescriptorRecord.from_xml_elem(elem)

In [ ]:
parse_mesh('../04-filtrage/MeSH/fredesc2019.xml')

<generator object parse_mesh at 0x0000025BCF0E0900>